In [8]:
include("../../src/test.jl")
include("../../src/forces.jl")
include("../../src/neighbor.jl")
include("../../src/run_event.jl")
include("../../src/forces_func.jl")
include("../../src/struct_data.jl")

Model

In [9]:
@make_struct_func Cubic

force_func (generic function with 1 method)

## Add Data

In [10]:
parameters = Model(
    Cubic(0.1,2,3),
    Contractile(0.0000001),
    Time(100000, 0.1),
    Neighbor(50,12),
    Geometry(9,2),
    Simulation(200)
)

Model(Cubic(0.1, 2.0, 3.0), Contractile(1.0e-7), Time(100000.0, 0.1), Neighbor(50, 12), Geometry(9.0, 2), Simulation(200))

In [17]:
R_agg = Int(parameters.ParGeometry.R_agg)
X = Float32.(readdlm("../../data/init/Sphere/$R_agg.xyz")[3:end,2:end]) |> cu

## Var before func
force(r) =  force_func(parameters.Force::Cubic, r)

force (generic function with 1 method)

## forces.jl -> cu_forces

In [19]:
function cui_force(t::Time, f::Contractile, n::Neighbor)
    # Definig Variables for calculing dX
    global X; global dX; global idx

    # Calculating distance for random forces (contractile)
    r_p = X - X[rand_idx[mod(t.t_f, size(X,1))+1, :], :]
    # Finding Distances/Norm for random forces
    dist_p = (sum(r_p .^ 2, dims=2).^ 0.5)

    # Finding distances
    r = reshape(repeat(X, inner=(n.nn,1)), n.nn, size(X)[1], 3) - X[getindex.(idx,1),:]

    # Finding Distances(Norm)
    dist = ((sum(r .^ 2, dims=3)) .^ 0.5)[:,:,1]

    # Finding forces for each cell
    F = force(dist) .* r ./ dist

    # Calculating de dX   -> dX[i,:] +=  r/dist * F
    dX = sum(F[2:end,:,:]; dims=1)[1,:,:] - f.fₚ .* (r_p ./ dist_p)
    synchronize()
end

cui_force (generic function with 1 method)